In [0]:
"""
https://www.youtube.com/watch?v=KWOZ9VoVFac&t=1s
If Criteria1, Criteria2 both are Y and a min of 2 team members should have Y then the output should be Y else N
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(
  [
    ('T1','T1_mbr1','Y','Y'),
    ('T1','T1_mbr2','Y','Y'),
    ('T1','T1_mbr3','Y','Y'),
    ('T1','T1_mbr4','Y','Y'),
    ('T1','T1_mbr5','Y','N'),
    ('T2','T2_mbr1','Y','Y'),
    ('T2','T2_mbr2','Y','N'),
    ('T2','T2_mbr3','N','Y'),
    ('T2','T2_mbr4','N','N'),
    ('T2','T2_mbr5','N','N'),
    ('T3','T3_mbr1','Y','Y'),
    ('T3','T3_mbr2','Y','Y'),
    ('T3','T3_mbr3','N','Y'),
    ('T3','T3_mbr4','N','Y'),
    ('T3','T3_mbr5','Y','N')
  ], [ "teamID", "memberID", "Criteria1", "Criteria2" ]
)

df.show()
df.printSchema()

+------+--------+---------+---------+
|teamID|memberID|Criteria1|Criteria2|
+------+--------+---------+---------+
|    T1| T1_mbr1|        Y|        Y|
|    T1| T1_mbr2|        Y|        Y|
|    T1| T1_mbr3|        Y|        Y|
|    T1| T1_mbr4|        Y|        Y|
|    T1| T1_mbr5|        Y|        N|
|    T2| T2_mbr1|        Y|        Y|
|    T2| T2_mbr2|        Y|        N|
|    T2| T2_mbr3|        N|        Y|
|    T2| T2_mbr4|        N|        N|
|    T2| T2_mbr5|        N|        N|
|    T3| T3_mbr1|        Y|        Y|
|    T3| T3_mbr2|        Y|        Y|
|    T3| T3_mbr3|        N|        Y|
|    T3| T3_mbr4|        N|        Y|
|    T3| T3_mbr5|        Y|        N|
+------+--------+---------+---------+

root
 |-- teamID: string (nullable = true)
 |-- memberID: string (nullable = true)
 |-- Criteria1: string (nullable = true)
 |-- Criteria2: string (nullable = true)



In [0]:

qualified_df = df.filter( (col("Criteria1") == "Y") & (col("Criteria2") == "Y") ) \
    .groupBy("teamID").agg(count("teamID").alias("total_qualified")) \
    .filter(col("total_qualified") >= 2)
qualified_df.show()


df.join(qualified_df, on="teamID", how="left") \
    .withColumn( "qualified_", when( (col("Criteria1") == "Y") & (col("Criteria2") == "Y") & (col("total_qualified").isNotNull()), "Y").otherwise("N") ) \
    .drop("total_qualified") \
    .show()

+------+---------------+
|teamID|total_qualified|
+------+---------------+
|    T1|              4|
|    T3|              2|
+------+---------------+

+------+--------+---------+---------+----------+
|teamID|memberID|Criteria1|Criteria2|qualified_|
+------+--------+---------+---------+----------+
|    T1| T1_mbr1|        Y|        Y|         Y|
|    T1| T1_mbr2|        Y|        Y|         Y|
|    T1| T1_mbr3|        Y|        Y|         Y|
|    T1| T1_mbr4|        Y|        Y|         Y|
|    T1| T1_mbr5|        Y|        N|         N|
|    T2| T2_mbr1|        Y|        Y|         N|
|    T2| T2_mbr2|        Y|        N|         N|
|    T2| T2_mbr3|        N|        Y|         N|
|    T2| T2_mbr4|        N|        N|         N|
|    T3| T3_mbr1|        Y|        Y|         Y|
|    T2| T2_mbr5|        N|        N|         N|
|    T3| T3_mbr2|        Y|        Y|         Y|
|    T3| T3_mbr3|        N|        Y|         N|
|    T3| T3_mbr4|        N|        Y|         N|
|    T3| T3_mbr5